# 📌 Notebook 1: Embed and Store Shot Data in Qdrant

This notebook loads your cleaned shot data, embeds it using `bge-small-en-v1.5`, and stores the embeddings in Qdrant for use in retrieval and recommendation.

# Initial Setup

In [4]:
# Step 1: Initial Setup:

# Load sentence-transformers and GTE-small model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('thenlper/gte-small')

## Load Shot Data From .csv

In [5]:
# Step 2: Load cleaned shot data
import pandas as pd
shot_data = pd.read_csv('../data/raw/cleaned_shot_data.csv')
shot_data.head()

,Date,Club Type,Club Description,Carry Distance,Total Distance,Ball Speed,Club Speed,Spin Rate,Attack Angle,Descent Angle,Shot Classification
0,2025-02-04 12:41:00,Driver,TopGolf - Driver (+1; N; 2.75T),124.33,171.19,122.16,85.92,1154,2.95,11.33,Hook
1,2025-02-04 12:41:42,Driver,TopGolf - Driver (+1; N; 2.75T),104.75,150.95,120.35,84.20,1666,2.45,8.19,Push Hook
2,2025-02-04 12:42:17,Driver,TopGolf - Driver (+1; N; 2.75T),163.45,195.51,115.05,86.28,1227,4.30,23.02,Push
3,2025-02-04 12:43:05,Driver,TopGolf - Driver (+1; N; 2.75T),162.57,192.56,110.91,81.96,1783,1.74,24.87,Push
4,2025-02-04 12:44:18,Driver,TopGolf - Driver (+1; N; 2.75T),105.30,152.00,118.83,80.78,1478,1.29,8.67,Push Draw


## Embed Shot Data

In [6]:
# Step 3: Format shot data into text chunks for embedding
def create_embedding_text(row):
    return f"{row['Club Type']} | Carry: {row['Carry Distance']} yds | Ball Speed: {row['Ball Speed']} mph | Classification: {row['Shot Classification']}"

texts = shot_data.apply(create_embedding_text, axis=1).tolist()

In [7]:
# Generate embeddings for shot text
embeddings = model.encode(texts, show_progress_bar=True)

Batches: 100%|██████████| 16/16 [00:17<00:00,  1.10s/it]


## Upload Embedded Shot Data to Qdrant

In [8]:
# Collect the Qdrant API key
from getpass import getpass

qdrant_api_key = getpass('🔑 Enter your Qdrant API Key: ')


In [11]:
# Qdrant setup
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, PointStruct, Distance

client = QdrantClient(
    url='https://6f592f43-f667-4234-ad3a-4f15ed5882ef.us-west-2-0.aws.cloud.qdrant.io:6333',
    api_key=qdrant_api_key
)

# Recreate the collection to flush old data
client.recreate_collection(
    collection_name='golf_shot_vectors',
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

/var/folders/5p/gq47dsys3k5663k1r5z8s3c40000gn/T/ipykernel_54911/3543225837.py:11: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [12]:
# Upload embedded vectors to Qdrant
points = [
    PointStruct(id=i, vector=embeddings[i], payload={'text': texts[i]})
    for i in range(len(embeddings))
]
client.upsert(collection_name='golf_shot_vectors', points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)